Market depth (order book)
==============

In [2]:
from ib_async import *

util.startLoop()

ib = IB()
ib.connect("127.0.0.1", 4001, clientId=10)
util.logToConsole()

In [ ]:
from IPython.display import display, clear_output
import pandas as pd

df = pd.DataFrame(index=range(5), columns="bidSize bidPrice askPrice askSize".split())

# NQM2024 contract
contract = Contract(conId=620730920)

ib.qualifyContracts(contract)
ticker = ib.reqMktDepth(contract)


def onTickerUpdate(ticker):
    bids = ticker.domBids
    for i in range(5):
        df.iloc[i, 0] = bids[i].size if i < len(bids) else 0
        df.iloc[i, 1] = bids[i].price if i < len(bids) else 0
    asks = ticker.domAsks
    for i in range(5):
        df.iloc[i, 2] = asks[i].price if i < len(asks) else 0
        df.iloc[i, 3] = asks[i].size if i < len(asks) else 0
    clear_output(wait=True)
    display(df)


ticker.updateEvent += onTickerUpdate

IB.sleep(15)

,bidSize,bidPrice,askPrice,askSize
0,2.0,17909.75,17910.5,2.0
1,3.0,17909.5,17910.75,3.0
2,2.0,17909.25,17911.0,4.0
3,7.0,17909.0,17911.25,4.0
4,3.0,17908.75,17911.5,7.0


True

In [ ]:
ib.cancelMktDepth(contract)

,bidSize,bidPrice,askPrice,askSize
0,2.0,17909.75,17910.5,2.0
1,3.0,17909.5,17910.75,4.0
2,3.0,17909.25,17911.0,3.0
3,7.0,17909.0,17911.25,3.0
4,4.0,17908.75,17911.5,3.0


In [13]:
ib.positions(account="U10394496")

[Position(account='U10394496', contract=Future(conId=620730920, symbol='NQ', lastTradeDateOrContractMonth='20240621', multiplier='20', currency='USD', localSymbol='NQM4', tradingClass='NQ'), position=-6.0, avgCost=357512.13917185),
 Position(account='U10394496', contract=Option(conId=669333316, symbol='NVDA', lastTradeDateOrContractMonth='20240719', strike=555.0, right='P', multiplier='100', currency='USD', localSymbol='NVDA  240719P00555000', tradingClass='NVDA'), position=1.0, avgCost=493.0494),
 Position(account='U10394496', contract=Option(conId=613471094, symbol='GOOGL', lastTradeDateOrContractMonth='20240920', strike=120.0, right='P', multiplier='100', currency='USD', localSymbol='GOOGL 240920P00120000', tradingClass='GOOGL'), position=10.0, avgCost=51.44685),
 Position(account='U10394496', contract=Option(conId=699532596, symbol='QQQ', lastTradeDateOrContractMonth='20240607', strike=400.0, right='P', multiplier='100', currency='USD', localSymbol='QQQ   240607P00400000', tradingC

In [45]:
def print_all_openorders(sym="NQ"):
    print("All Open Orders")
    print(f"symbol\tpermId\t\tstatus\t\taction\tfilled\tremaining\tlmtPrice")

    trades = ib.reqAllOpenOrders()

    trades.sort(key=lambda trade: trade.order.lmtPrice)

    for trade in trades:
        orderstatus = trade.orderStatus
        order = trade.order

        if trade.contract.symbol != sym:
            continue

        print(
            f"{trade.contract.symbol}\t{order.permId}\t{orderstatus.status}\t{order.action}\t{orderstatus.filled}\t{orderstatus.remaining}\t\t{order.lmtPrice}\t"
        )


def print_trades(trades):
    print(f"symbol\tpermId\t\tstatus\t\taction\tfilled\tremaining\tlmtPrice")

    trades.sort(key=lambda trade: trade.order.lmtPrice)

    for trade in trades:
        orderstatus = trade.orderStatus
        order = trade.order

        print(
            f"{trade.contract.symbol}\t{order.permId}\t{orderstatus.status}\t{order.action}\t{orderstatus.filled}\t{orderstatus.remaining}\t\t{order.lmtPrice}\t"
        )


def get_all_openorders(sym="NQ"):
    trades = ib.reqAllOpenOrders()
    trades.sort(key=lambda trade: trade.order.lmtPrice)

    trades = [trade for trade in trades if trade.contract.symbol == sym]
    return trades


# print_all_openorders()

In [31]:
def print_order(o):
    order = o.order
    contract = o.contract
    orderStatus = o.orderStatus

    print(f"symbol\tpermId\t\tstatus\t\taction\tfilled\tremaining\tlmtPrice")

    print(
        f"{contract.symbol}\t{order.permId}\t{orderStatus.status}\t{order.action}\t{orderStatus.filled}\t{orderStatus.remaining}\t\t{order.lmtPrice}\t"
    )


print_order(takeprofit)

symbol	permId		status		action	filled	remaining	lmtPrice
NQ	740241587	Submitted	SELL	0.0	1.0		17912.25	


In [11]:
openorders = get_all_openorders()
print([o for o in openorders if o.order.permId == 740241587])

[Trade(contract=Future(conId=620730920, symbol='NQ', lastTradeDateOrContractMonth='20240621', right='?', multiplier='20', exchange='CME', currency='USD', localSymbol='NQM4', tradingClass='NQ'), order=Order(permId=740241587, action='SELL', totalQuantity=1.0, orderType='LMT', lmtPrice=17912.25, auxPrice=0.0, tif='GTC', ocaGroup='740241586', ocaType=3, parentId=2147483647, displaySize=2147483647, rule80A='0', openClose='', volatilityType=0, deltaNeutralOrderType='None', referencePriceType=0, account='U10394496', clearingIntent='IB', adjustedOrderType='None', cashQty=0.0, dontUseAutoPriceForHedge=True), orderStatus=OrderStatus(orderId=0, status='Submitted', filled=0.0, remaining=1.0, avgFillPrice=0.0, permId=740241587, parentId=2147483647, lastFillPrice=0.0, clientId=0, whyHeld='', mktCapPrice=0.0), fills=[], log=[TradeLogEntry(time=datetime.datetime(2024, 4, 29, 6, 40, 29, 121688, tzinfo=datetime.timezone.utc), status='Submitted', message='', errorCode=0)], advancedError='')]


In [44]:
# L1 = buy @ mid - 10 tickets , tp + 10 ticks

qty_target = -7
nq24 = Contract(conId=620730920)

# get current position of contract nq24
position = ib.positions(account="U10394496")
future = position[0]

openorders = get_all_openorders(sym="NQ")

# seed parameters
opentobuy = None
takeprofit = [o for o in openorders if o.order.permId == 740241587][0]

print("Open to Buy Order:")
print(opentobuy)
print()
print("Take Profit Order:")
print_order(takeprofit)
print()

if future.position == qty_target:
    print("Position is at target quantity / {}, {}".format(future.position, qty_target))
    # check for initiation order (L1.buy)
    if opentobuy is None:
        print("Initiation Order does not exist")
        print()
        # get current market price
        
        # place L1.buy
    else:
        print("Initiation Order exists...")
        print()

if future.position > qty_target:
    print("future.position {} > {} qty_target".format(future.position, qty_target))
    # check for take profit order (L1.sell)
    if takeprofit is None:
        print("Take Profit Order does not exist")
        print()
        # place L1.sell
    else:
        print("Take Profit Order exists...")
        print()

Open to Buy Order:
None

Take Profit Order:
symbol	permId		status		action	filled	remaining	lmtPrice
NQ	740241587	Submitted	SELL	0.0	1.0		17912.25	

future.position -6.0 > -7 qty_target
Take Profit Order exists...



In [46]:
ticker

NameError: name 'ticker' is not defined

In [37]:
def print_all_openorders(sym="NQ"):
    print("All Open Orders:\n")
    print(f"symbol\tpermId\t\tstatus\t\taction\tfilled\tremaining\tlmtPrice")

    trades = ib.reqAllOpenOrders()

    # order every Trade object by its order.lmtPrice field
    trades.sort(key=lambda trade: trade.order.lmtPrice)

    for trade in trades:
        orderstatus = trade.orderStatus
        order = trade.order

        if trade.contract.symbol != sym:
            continue

        print(
            f"{trade.contract.symbol}\t{order.permId}\t{orderstatus.status}\t{order.action}\t{orderstatus.filled}\t{orderstatus.remaining}\t\t{order.lmtPrice}\t"
        )


def get_all_openorders(sym="NQ"):
    trades = ib.reqAllOpenOrders()
    trades.sort(key=lambda trade: trade.order.lmtPrice)

    # or return trades where contract.conId == 620730920
    trades = [trade for trade in trades if trade.contract.symbol == sym]
    return trades


trades = get_all_openorders()

print_all_openorders()

All Open Orders:

symbol	permId		status		action	filled	remaining	lmtPrice
NQ	1528433802	Submitted	BUY	0.0	3.0		17526.0	
NQ	1436152242	Submitted	BUY	0.0	1.0		17547.25	
NQ	1786380178	Submitted	BUY	0.0	1.0		17623.25	
NQ	1786380182	Submitted	BUY	0.0	1.0		17650.0	
NQ	740241587	Submitted	SELL	0.0	1.0		17912.25	
NQ	1056387955	PreSubmitted	SELL	0.0	2.0		20880.0	


In [47]:
openOrders = ib.reqAllOpenOrders()
for order in openOrders:
    if order.contract.conId == 620730920:
        print(order)

# print(openOrders)

Trade(contract=Future(conId=620730920, symbol='NQ', lastTradeDateOrContractMonth='20240621', right='?', multiplier='20', exchange='CME', currency='USD', localSymbol='NQM4', tradingClass='NQ'), order=Order(permId=1528433802, action='BUY', totalQuantity=3.0, orderType='LMT', lmtPrice=17526.0, auxPrice=0.0, tif='GTC', ocaGroup='1528433801', ocaType=3, displaySize=2147483647, rule80A='0', openClose='', volatilityType=0, deltaNeutralOrderType='None', referencePriceType=0, account='U10394496', clearingIntent='IB', adjustedOrderType='None', cashQty=0.0, dontUseAutoPriceForHedge=True), orderStatus=OrderStatus(orderId=0, status='Submitted', filled=0.0, remaining=3.0, avgFillPrice=0.0, permId=1528433802, parentId=0, lastFillPrice=0.0, clientId=0, whyHeld='', mktCapPrice=0.0), fills=[], log=[TradeLogEntry(time=datetime.datetime(2024, 4, 29, 2, 30, 39, 407388, tzinfo=datetime.timezone.utc), status='Submitted', message='', errorCode=0)], advancedError='')
Trade(contract=Future(conId=620730920, symb

In [35]:
# canceled orders
print(f"permId\t\taction\t\ttotalQuantity\tfilledQuantity\t\tlmtPrice")

closedOrders = ib.orders()
[
    print(
        f"{order.permId}\t{order.action}\t\t{order.totalQuantity}\t\t{order.filledQuantity}\t\t\t{order.lmtPrice}"
    )
    for order in closedOrders
    if order.filledQuantity == 0
]

permId		action		totalQuantity	filledQuantity		lmtPrice
986847725	BUY		1.0		0.0			17875.0
986847726	SELL		1.0		0.0			17877.5
986847719	BUY		1.0		0.0			17862.25
986847720	SELL		1.0		0.0			17865.75
986847669	BUY		1.0		0.0			17855.0
986847670	SELL		1.0		0.0			17857.5


[None, None, None, None, None, None]

In [36]:
ib.reqOpenOrders()
ib.sleep(15)

KeyboardInterrupt: 

In [6]:
ib.disconnect()